In [4]:
from unmixing_multithreading import task, p_alive, SpectralUnmixer
from utility import make_dir
import os
import pandas as pd
import time
import copy
import tensorflow as tf
from multiprocessing import Process
from tqdm.notebook import tqdm, trange

# Data Source

In [5]:
SUB_DIR = "SI data (1)\\"
MAIN_DIR = "C:\\Users\\KoegelMarco\\Documents\\Ergebnisse\\DataShare\\PhD\\Li Unmixing Test\\"
PREPROCESS_MAIN_DIR = MAIN_DIR + "preprocess_tuning_Li-edge_no-binning\\"
folder = "SI data (1)"

# Specimen properties

In [6]:
ELEMENTS = ["Li"]  # only for background removal

COMP_NAMES_ALL = ["comp" + str(i) for i in range(100)]

parameters = {"component_names": [""],   # Placeholder
              "omegas": [None, 1, None],     # Datafusion weighting [EELS_LL, EELS_CL, EDX]
              "end_eels_cl": ("custom", 52, 80)}   # ("custom", start_EELS_energy[eV], end_EELS_energy[eV[]]

# Learning parameter space

In [7]:
Rs = [4, 5, 6, 7, 8, 9, 10, 11]#,7,8,9]
batch_sizes = [20, 50, 100]#, 10, 300]
epochs = [500, 500]#, 400] # additive
learning_rates = [0.005]
dropouts = [0.01]
layer_activations = ["sigmoid"]
end_activations = [("sig", tf.keras.activations.sigmoid)]
ab_rates = [1e-2, 1e-1, 1, 10]
neg_rates = [1e-1]
layer_unitss = [[9, 6, 3, 1]]#, [16, 8, 4, 1]]

# Hardware Specs

In [8]:
N_CPU = 4

# Preprocessing

In [9]:
unmixer = SpectralUnmixer(ELEMENTS, parameters)


print("-----------------------------")
print(folder)
print("-----------------------------")

make_dir(PREPROCESS_MAIN_DIR + SUB_DIR)

sample = MAIN_DIR
splits = sample.split("\\")
#main = splits[-4] + "\\"
sample = folder + "\\"
print("PREPROCESS data file in " + sample)
make_dir(PREPROCESS_MAIN_DIR + sample)


unmixer.add_data(MAIN_DIR + sample, remove_eels_background=False, use_refs=False, binning_factor=1)


unmixer.shuffle()




if not os.path.isfile(PREPROCESS_MAIN_DIR + "tuning.pkl") :
    df = pd.DataFrame(columns=["R", "learning_rate", "end_activation",
                                          "layer_activation", "ab_rate", "neg_rate", "layer_units",
                                          "dropout", "batch_size", "epoch",
                                          "loss",
                                          "error", "loss_pred", "err_pred", "adundance_diff", "spec_dist"])
    df.to_pickle(PREPROCESS_MAIN_DIR + "tuning.pkl")
    N_start = 0
else:
    df = pd.read_pickle(PREPROCESS_MAIN_DIR + "tuning.pkl")
    N_start = len(df.index) + 1

N_curr = N_start

save_name=None
#comp_names = parameters["component_names"]
fusion_data_binned = unmixer.data_collection

pre_predicter = unmixer.make_predicter()
pre_predicter.stuff(use_refs=False, PATH=MAIN_DIR + sample, remove_eels_background=False)
predicter = pre_predicter.make_predicter()
p_list = []

for R in Rs:
    predicter = copy.copy(predicter)
    parameters = {"component_names": COMP_NAMES_ALL[:R],
                  "omegas": [None, 1, None],
                  "end_eels_cl": ("custom", 52,80)}
    predicter.parameters = parameters
    for learning_rate in learning_rates:
        for end_act_name, end_activation in end_activations:
            for layer_activation in layer_activations:
                for ab_rate in ab_rates:
                    for neg_rate in neg_rates:
                        for layer_units in layer_unitss:
                            for dropout in dropouts:
                                for batch_size in batch_sizes:
                                    data_pack = (MAIN_DIR + sample, PREPROCESS_MAIN_DIR, predicter, fusion_data_binned, save_name)
                                    process = Process(target=task, args=(N_curr, data_pack, R, learning_rate, (end_act_name, end_activation),
                                                        layer_activation, layer_units, ab_rate, neg_rate,
                                                        dropout, batch_size, epochs,None))
                                    p_list.append(process)
                                    N_curr += len(epochs)
                                    #process.start()




-----------------------------
SI data (1)
-----------------------------
PREPROCESS data file in SI data (1)\


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	7.22
std:	2.18

min:	2
Q1:	5.5
median:	7.25
Q3:	8.75
max:	12.5
[########################################] | 100% Completed | 2.08 ss


[########################################] | 100% Completed | 2.06 ss
[########################################] | 100% Completed | 1.63 ss
[########################################] | 100% Completed | 766.33 ms


[########################################] | 100% Completed | 760.95 ms


[########################################] | 100% Completed | 282.67 ms


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	7.22
std:	2.18

min:	2
Q1:	5.5
median:	7.25
Q3:	8.75
max:	12.5
[########################################] | 100% Completed | 2.18 ss


[########################################] | 100% Completed | 2.07 ss
[########################################] | 100% Completed | 1.65 ss
[########################################] | 100% Completed | 765.48 ms


[########################################] | 100% Completed | 761.07 ms


[########################################] | 100% Completed | 364.26 ms


# Start unmixing (multithreading)

In [ ]:
for p in tqdm(p_list):
    while p_alive(p_list) >= N_CPU:
        time.sleep(3)
    p.start()
    #print("process " + p.name + " started")
    #print(str(p_alive(p_list)) + " processes active")
    #print("process finished")

  0%|          | 0/96 [00:00<?, ?it/s]

# Results

In [10]:
df = pd.read_pickle(PREPROCESS_MAIN_DIR + "tuning.pkl")
df.sort_values("error")

,R,learning_rate,end_activation,layer_activation,ab_rate,neg_rate,layer_units,dropout,batch_size,epoch,loss,error,loss_pred,err_pred,adundance_diff,spec_dist
172,11,0.005,sig,sigmoid,0.01,0.1,"[9, 6, 3, 1]",0.01,100,500,0.322961,0.560910,0.324863,0.560909,0.713903,0.111825
148,10,0.005,sig,sigmoid,0.01,0.1,"[9, 6, 3, 1]",0.01,100,500,0.322767,0.566207,0.324610,0.566207,0.642769,0.108254
124,9,0.005,sig,sigmoid,0.01,0.1,"[9, 6, 3, 1]",0.01,100,500,0.325127,0.569652,0.326267,0.569652,0.499031,0.083870
100,8,0.005,sig,sigmoid,0.01,0.1,"[9, 6, 3, 1]",0.01,100,500,0.323449,0.582805,0.324930,0.582805,0.589215,0.090716
76,7,0.005,sig,sigmoid,0.01,0.1,"[9, 6, 3, 1]",0.01,100,500,0.321816,0.590202,0.322880,0.590202,0.536111,0.068945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,5,0.005,sig,sigmoid,10.00,0.1,"[9, 6, 3, 1]",0.01,20,1000,310.107666,37.196274,310.107666,37.196274,0.203805,1.679050
187,11,0.005,sig,sigmoid,10.00,0.1,"[9, 6, 3, 1]",0.01,20,1000,309.997162,37.898888,309.997162,37.898888,0.096971,1.065368
61,6,0.005,sig,sigmoid,1.00,0.1,"[9, 6, 3, 1]",0.01,20,1000,31.114397,38.278500,31.114397,38.278500,0.308206,3.646057
55,6,0.005,sig,sigmoid,0.10,0.1,"[9, 6, 3, 1]",0.01,20,1000,3.118925,39.153919,3.118925,39.153919,0.374356,2.068398
